# Etape 1.1 : Exploration initiale

**Livrables** :
- Notebook `01_exploration_spark.ipynb`
- Rapport d'audit (format markdown ou DataFrame - `01_rapport_audit_donnees.md`)

---
---

## Imports

In [1]:
import os
import sys
import platform
from pathlib import Path
import psutil
import time
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

---

## (optionnel) Enregistrement de la date de la dernière execution de ce notebook

In [2]:
print(f"- Date de la dernière execution de ce notebook : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} (FR)")

- Date de la dernière execution de ce notebook : 20/02/2026 20:29:35 (FR)


---

## (Optionnel) Mesure du temps de traitement global pour ce script - enregistrement de l'heure de début + estimation instantanée des ressources machine libres

In [3]:
# --- Heure de début
start_time_01 = time.time()

# --- Machine: current available RAM (in GB)
ram_available_01 = psutil.virtual_memory().available / (1024**3)

# --- Machine: current available CPU
logical = psutil.cpu_count()
physical = psutil.cpu_count(logical=False) or logical

cpu_used = psutil.cpu_percent(interval=2)
cpu_available_pct_01 = 100 - cpu_used

available_logical_01 = logical * cpu_available_pct_01 / 100
available_physical_01 = physical * cpu_available_pct_01 / 100

# --- Show available resources
print(f"- Current machine RAM available : {ram_available_01:.2f} GB")
print(f"- Current machine CPU available : {cpu_available_pct_01:.1f}%")
print(f"    Approx logical cores free  : {available_logical_01:.2f}")
print(f"    Approx physical cores free : {available_physical_01:.2f}")


- Current machine RAM available : 11.09 GB
- Current machine CPU available : 84.3%
    Approx logical cores free  : 13.49
    Approx physical cores free : 6.74


---

## Chemins des données

In [4]:
# ==============================================================================================================
#                                                  INPUTS
# ==============================================================================================================
IN_DIR = (Path.cwd() / ".." / "data").resolve()
IN_CONSO_RAW_CSV = os.path.join(IN_DIR, "consommations_raw-test.csv")
#IN_CONSO_RAW_CSV = os.path.join(IN_DIR, "consommations_raw.csv")
IN_BAT_CSV = os.path.join(IN_DIR, "batiments.csv")

# ==============================================================================================================
#                                                  OUTPUTS
# ==============================================================================================================
OUT_DIR = (Path.cwd() / ".." / "output").resolve()
OUT_RAPPORT_AUDIT_MD = os.path.join(OUT_DIR, "01_rapport_audit_donnees.md")

# ==============================================================================================================
#                                                  OTHERS
# ==============================================================================================================
TMP_DIR = (Path.cwd() / ".." / "my_tmp").resolve()
TMP_FILE_TXT = TMP_DIR / "tmp_01_resources.txt" # Enregistrer les metrics pour ce script

---

## Création d'une session Spark locale

In [5]:
# --- Creer une session Spark locale
spark = SparkSession.builder \
    .appName("ECF 2 - ENERGIE - Etape 1.1 : Exploration initiale") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

# --- Reduire les logs
spark.sparkContext.setLogLevel("WARN")

# --- Affichage de l'actuelle version de Spark & de l'url de Spark UI ainsi que python & java
print(f"Spark version  : {spark.version}")
print(f"Spark UI       : {spark.sparkContext.uiWebUrl}")
print()
print(f"Python version : {platform.python_version()}")
print(f"Python path    : { sys.executable}")
print()
print(f"Java version   : {spark._jvm.java.lang.System.getProperty('java.version')}")
print(f"Java home      : { spark._jvm.java.lang.System.getProperty('java.home')}")

Spark version  : 3.5.7
Spark UI       : http://joel:4041

Python version : 3.11.9
Python path    : C:\Users\joel\Downloads\ecf_energie\.venv\Scripts\python.exe

Java version   : 11.0.28
Java home      : C:\Users\joel\data-info\dev-tools\languages\java\jdk-11


---

## Chargement des donnees de consommation avec PySpark

In [6]:
# --- Chargement du CSV avec inference de schema
df_conso_raw = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(IN_CONSO_RAW_CSV)

print(f"- Les données de consommation ont été chargées avec succès: {IN_CONSO_RAW_CSV}")

# --- Information sur le nombre de lignes et de colonnes
print(f"- Nombre de lignes : {df_conso_raw.count():,}")
print("- Nombre de colonnes :", len(df_conso_raw.columns))

# --- Apercu des donnees
print("- Apercu des donnees :")
df_conso_raw.show(10, truncate=False)

- Les données de consommation ont été chargées avec succès: C:\Users\joel\Downloads\ecf_energie\data\consommations_raw-test.csv


- Nombre de lignes : 1,000
- Nombre de colonnes : 5
- Apercu des donnees :
+-----------+-------------------+------------+------------+-----+
|batiment_id|timestamp          |type_energie|consommation|unite|
+-----------+-------------------+------------+------------+-----+
|BAT0141    |2023-12-21 13:00:00|gaz         |342.34      |kWh  |
|BAT0080    |08/08/2023 13:00   |gaz         |1256.73     |kWh  |
|BAT0122    |06/13/2024 11:00:00|eau         |133.57      |m3   |
|BAT0033    |2023-06-25 00:00:00|eau         |0.23        |m3   |
|BAT0064    |11/29/2024 04:00:00|gaz         |12.26       |kWh  |
|BAT0052    |09/15/2024 18:00:00|gaz         |701.01      |kWh  |
|BAT0097    |2024-01-25 12:00:00|gaz         |444.87      |kWh  |
|BAT0073    |27/03/2023 13:00   |electricite |123.97      |kWh  |
|BAT0084    |03/12/2024 17:00:00|eau         |17.88       |m3   |
|BAT0022    |2024-10-30 03:00:00|eau         |0.80        |m3   |
+-----------+-------------------+------------+------------+-----+
o

---

## Analyse de schema infere et identification des problemes de typage

* **Schema infere**

In [7]:
df_conso_raw.printSchema()

root
 |-- batiment_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- type_energie: string (nullable = true)
 |-- consommation: string (nullable = true)
 |-- unite: string (nullable = true)



* **Analyse du schema infere & identification des problemes de typage**

Ce shema révèle que les données des deux colonnes ``timestamp`` & ``consommation`` ont besoin de transformations pour corriger leurs types :

- ``timestamp`` deviendera (après les futures nécéssaires transformations) de type timestamp ;
- ``consommation`` deviendera (après les futures nécéssaires transformations) de type double.

*timestamp :*

Problème : La forme des ``timestamp`` n'est pas unifiée :

In [8]:
print("Exemples de formats de timestamp:")
df_conso_raw.select("timestamp").distinct().show(10, truncate=False)

Exemples de formats de timestamp:


+-------------------+
|timestamp          |
+-------------------+
|2023-12-21 13:00:00|
|11/29/2024 04:00:00|
|09/15/2024 18:00:00|
|20/12/2023 07:00   |
|29/04/2024 13:00   |
|2024-06-15T14:00:00|
|22/03/2024 11:00   |
|2023-04-07 13:00:00|
|2024-03-11T11:00:00|
|08/21/2024 17:00:00|
+-------------------+
only showing top 10 rows



*consommation :*

Problème : elle contient des valeurs "non double" :

In [9]:
# --- Les valeurs qui ne peuvent pas etre converties en nombre
df_conso_non_numeric = df_conso_raw.filter(
    ~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")
)

nbre_vals_conso_non_numeric = df_conso_non_numeric.count() 

if nbre_vals_conso_non_numeric == 0:
    print("- La colonne 'consommation' ne contient pas de valeurs non numeriques")
else:
    print(f"- Nombre de valeurs non numeriques (non convertible en nombre) : {nbre_vals_conso_non_numeric:,}")
    print(f"- Differents valeurs non numeriques (non convertible en nombre) :")
    if df_conso_non_numeric.select("consommation").distinct().count() > 20:
        print("    Examples :")
        df_conso_non_numeric.select("consommation").distinct().show()
    else:
        df_conso_non_numeric.select("consommation").distinct().show()

- Nombre de valeurs non numeriques (non convertible en nombre) : 4
- Differents valeurs non numeriques (non convertible en nombre) :


+------------+
|consommation|
+------------+
|        null|
|         N/A|
|         ---|
+------------+



In [10]:
# --- Les valeurs qui peuvent etre converties en nombre => valeurs avec virgule comme separateur decimal
df_with_comma = df_conso_raw.filter(F.col("consommation").contains(","))

nbre_vals_with_comma = df_with_comma.count() 

if nbre_vals_with_comma == 0:
    print("- La colonne 'consommation' ne contient pas de valeurs avec des virgules décimales")
else:
    print(f"- Nombre de valeurs avec virgule (convertible en nombre) : {nbre_vals_with_comma:,}")
    # --- Affichages d'exemples
    print(f"- Exemples :")
    df_with_comma.select("consommation").show(5)

- Nombre de valeurs avec virgule (convertible en nombre) : 135
- Exemples :


+------------+
|consommation|
+------------+
|       10,10|
|       72,29|
|       17,82|
|        0,92|
|      290,65|
+------------+
only showing top 5 rows



---

## Calculer les statistiques descriptives par type d'energie

Initialement on a la colonne ``consommation`` en string. 

Pour calculer les statistiques descriptives, on va créer une dataFrame 'df_conso_numeric' qui contient les mêmes données que 'df_conso_raw' plus une colonne ``consommation_clean`` qui contient les valeurs de la colonne 'consommation' converties en double.

In [11]:
# --- Nombre des valeurs Nulls initiales dans df_conso_raw
nbre_conso_nulles = df_conso_raw.filter(F.col("consommation").isNull()).count()
print("- Nombre de valeurs nulles dans la colonne 'consommation' de la df_conso_raw :", nbre_conso_nulles)

# --- Ajouter de la colonne 'consommation_clean' dans la dataFrame df_conso_raw
#     & convertion de value en double (en remplacant la virgule par un point)
df_conso_numeric = df_conso_raw.withColumn(
    "consommation_clean",
    F.regexp_replace(F.col("consommation"), ",", ".").cast("double")
)
# ==> ici après le cast en double il n'y aura pas de valeurs non numérique dans "consommation_clean"
#     ils ont été transformé en null.

# --- Apercu de la df_conso_numeric
print("- Apercu de la df_conso_numeric :")
df_conso_numeric.show(10)

# --- Nombre des valeurs Nulls dans df_conso_numeric
print(f"- Nombre des valeurs Nulls dans la colonne 'consommation_clean' : {df_conso_numeric.filter(F.col('consommation_clean').isNull()).count():,}")

# --- Calcul des statistiques descriptives par type d'energie (en ignorant les valeurs nulles de 'consommation_clean')
stats_by_type_energie = df_conso_numeric.filter(F.col("consommation_clean").isNotNull()) \
    .groupBy("type_energie") \
    .agg(
        F.count("*").alias("count"),
        F.round(F.mean("consommation_clean"), 2).alias("mean"),
        F.round(F.stddev("consommation_clean"), 2).alias("stddev"),
        F.round(F.min("consommation_clean"), 2).alias("min"),
        F.round(F.max("consommation_clean"), 2).alias("max"),
        F.round(F.expr("percentile(consommation_clean, 0.5)"), 2).alias("median")
    ) \
    .orderBy("type_energie")

print("\n- Statistiques par type d'energie (en ignorant les valeurs nulles) :")
stats_by_type_energie.show()


- Nombre de valeurs nulles dans la colonne 'consommation' de la df_conso_raw : 0
- Apercu de la df_conso_numeric :


+-----------+-------------------+------------+------------+-----+------------------+
|batiment_id|          timestamp|type_energie|consommation|unite|consommation_clean|
+-----------+-------------------+------------+------------+-----+------------------+
|    BAT0141|2023-12-21 13:00:00|         gaz|      342.34|  kWh|            342.34|
|    BAT0080|   08/08/2023 13:00|         gaz|     1256.73|  kWh|           1256.73|
|    BAT0122|06/13/2024 11:00:00|         eau|      133.57|   m3|            133.57|
|    BAT0033|2023-06-25 00:00:00|         eau|        0.23|   m3|              0.23|
|    BAT0064|11/29/2024 04:00:00|         gaz|       12.26|  kWh|             12.26|
|    BAT0052|09/15/2024 18:00:00|         gaz|      701.01|  kWh|            701.01|
|    BAT0097|2024-01-25 12:00:00|         gaz|      444.87|  kWh|            444.87|
|    BAT0073|   27/03/2023 13:00| electricite|      123.97|  kWh|            123.97|
|    BAT0084|03/12/2024 17:00:00|         eau|       17.88|   m3|

- Nombre des valeurs Nulls dans la colonne 'consommation_clean' : 4



- Statistiques par type d'energie (en ignorant les valeurs nulles) :


+------------+-----+------+-------+-----+--------+------+
|type_energie|count|  mean| stddev|  min|     max|median|
+------------+-----+------+-------+-----+--------+------+
|         eau|  336|176.48|2344.08|-6.64|42978.44|  7.94|
| electricite|  347|354.96|2029.68| 1.24| 37327.8| 93.88|
|         gaz|  313|593.52|2461.41| 0.65| 33547.6|170.75|
+------------+-----+------+-------+-----+--------+------+



In [12]:
# --- Identification des valeurs négatives & aberrantes
print("- Valeurs negatives :")
df_conso_numeric.filter(F.col("consommation_clean") < 0).groupBy("type_energie").count().show()

print("- Valeurs abberantes > 15000 :")
df_conso_numeric.filter(F.col("consommation_clean") > 15000).groupBy("type_energie").count().show()


- Valeurs negatives :


+------------+-----+
|type_energie|count|
+------------+-----+
|         eau|    1|
+------------+-----+

- Valeurs abberantes > 15000 :


+------------+-----+
|type_energie|count|
+------------+-----+
|         eau|    1|
|         gaz|    2|
| electricite|    1|
+------------+-----+



---

## Identifier les batiments avec le plus de mesures

In [13]:
# --- Batiments ids avec le plus de mesures (affichage par ordre décroissant du nombre de mesures)
print("- Les batiments ids avec le plus de mesures (odre décroissant + affichage de seulement les 10 premiers) :")
df_bats_mesures = df_conso_raw.groupBy('batiment_id').count().withColumnRenamed("count", "nbre_de_mesures").orderBy(F.col("count").desc())
df_bats_mesures.show(10) 

- Les batiments ids avec le plus de mesures (odre décroissant + affichage de seulement les 10 premiers) :


+-----------+---------------+
|batiment_id|nbre_de_mesures|
+-----------+---------------+
|    BAT0117|             13|
|    BAT0082|             13|
|    BAT0074|             13|
|    BAT0052|             13|
|    BAT0059|             12|
|    BAT0093|             11|
|    BAT0011|             11|
|    BAT0121|             11|
|    BAT0071|             11|
|    BAT0003|             11|
+-----------+---------------+
only showing top 10 rows



In [14]:
# --- Chargement du CSV des batiments avec inference de schema
df_bat = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(IN_BAT_CSV)

print(f"- Fichier de des batiments a été chargé avec succès : {IN_BAT_CSV}")

# --- Information sur le nombre de lignes et de colonnes
print("- Nombre de lignes :", df_bat.count())
print("- Nombre de colonnes :", len(df_bat.columns))

# --- Apercu des donnees
print("- Apercu des donnees :")
df_bat.show(10, truncate=False)

- Fichier de des batiments a été chargé avec succès : C:\Users\joel\Downloads\ecf_energie\data\batiments.csv


- Nombre de lignes : 146
- Nombre de colonnes : 8
- Apercu des donnees :
+-----------+-------------------+-----------+-------+----------+------------------+------------------+------------------+
|batiment_id|nom                |type       |commune|surface_m2|annee_construction|classe_energetique|nb_occupants_moyen|
+-----------+-------------------+-----------+-------+----------+------------------+------------------+------------------+
|BAT0001    |Ecole Paris 1      |ecole      |Paris  |1926      |1978              |E                 |225               |
|BAT0002    |Ecole Paris 2      |ecole      |Paris  |1156      |2004              |C                 |402               |
|BAT0003    |Ecole Paris 3      |ecole      |Paris  |1695      |2014              |D                 |219               |
|BAT0004    |Mediatheque Paris 4|mediatheque|Paris  |907       |2019              |C                 |121               |
|BAT0005    |Piscine Paris 5    |piscine    |Paris  |3913      |1950     

In [15]:
# --- Affichage des batiments (avec leurs données) avec le plus de mesures (ordre décroissant)
df_bats_mesures.join(df_bat, on="batiment_id", how="left").orderBy(F.col("nbre_de_mesures").desc()).show()

+-----------+---------------+--------------------+-----------+-------------+----------+------------------+------------------+------------------+
|batiment_id|nbre_de_mesures|                 nom|       type|      commune|surface_m2|annee_construction|classe_energetique|nb_occupants_moyen|
+-----------+---------------+--------------------+-----------+-------------+----------+------------------+------------------+------------------+
|    BAT0117|             13| Mairie Le Havre 117|     mairie|     Le Havre|      1257|              2018|                 A|                36|
|    BAT0082|             13|Gymnase Montpelli...|    gymnase|  Montpellier|      1719|              2015|                 C|               214|
|    BAT0074|             13|Mairie Strasbourg 74|     mairie|   Strasbourg|      1180|              1955|                 F|                36|
|    BAT0052|             13|    Piscine Lille 52|    piscine|        Lille|      2668|              2012|                 B|     

---

## Production d'un rapport d'audit de qualite des donnees

In [16]:
# --- Nombre total des enregistrements
total_enregistrements = df_conso_raw.count()

# --- Valeurs non numeriques
conso_non_numeric = df_conso_raw.filter(
    ~F.col("consommation").rlike("^-?[0-9]+[.,]?[0-9]*$")
).count()

# --- Valeurs avec virgule
with_comma = df_conso_raw.filter(F.col("consommation").contains(",")).count()

# --- Valeurs negatives (apres conversion)
negative = df_conso_numeric.filter(F.col("consommation_clean") < 0).count()

# --- Valeurs aberrantes > 15000
outliers = df_conso_numeric.filter(F.col("consommation_clean") > 15000).count()

# --- Doublons
duplicates = total_enregistrements - df_conso_raw.dropDuplicates(["batiment_id", "timestamp", "type_energie"]).count()

# --- Optional : Temps d'execution de ce script + ressources allouées au départ
temps_execution_01 = time.time() - start_time_01

# --- Generer le rapport markdown
rapport_audit = f"""# Rapport d'audit qualité des données de consommation énergétique des bâtiments

# --- Total enregistrements
{total_enregistrements:,}

# --- Problemes identifies
- Valeurs non numeriques : {conso_non_numeric:,} ({conso_non_numeric/total_enregistrements*100:.2f}%)
- Valeurs avec virgule decimale : {with_comma:,} ({with_comma/total_enregistrements*100:.2f}%)
- Valeurs negatives : {negative:,} ({negative/total_enregistrements*100:.2f}%)
- Valeurs aberrantes (>15000) : {outliers:,} ({outliers/total_enregistrements*100:.2f}%)
- Doublons : {duplicates:,} ({duplicates/total_enregistrements*100:.2f}%)
- Formats de dates multiples : 4 formats differents detectes

---
*Rapport genere automatiquement - Audit de qualite des donnees de consommation énergetique des batiments*  
*Données brutes : consommations_raw.csv*  
*Script de traitement : 01_exploration_spark.ipynb*  
*Temps de traitement : {temps_execution_01:.2f} secondes*  
*Ressources machine approximatives allouées à ce traitement : (RAM : {ram_available_01:.2f} GB, CPU : {cpu_available_pct_01:.2f}% - Approx logical cores free({available_logical_01:.2f}) - Approx physical cores free : {available_physical_01:.2f})*  
*Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)* 
"""

# Sauvegarder le rapport d'audit
with open(f"{OUT_RAPPORT_AUDIT_MD}", 'w', encoding='utf-8') as f:
    f.write(rapport_audit)

print(f"Rapport d'audit sauvegarde : {OUT_RAPPORT_AUDIT_MD}")
print("\n" + "="*60)
print(rapport_audit)


Rapport d'audit sauvegarde : C:\Users\joel\Downloads\ecf_energie\output\01_rapport_audit_donnees.md

# Rapport d'audit qualité des données de consommation énergétique des bâtiments

# --- Total enregistrements
1,000

# --- Problemes identifies
- Valeurs non numeriques : 4 (0.40%)
- Valeurs avec virgule decimale : 135 (13.50%)
- Valeurs negatives : 1 (0.10%)
- Valeurs aberrantes (>15000) : 4 (0.40%)
- Doublons : 0 (0.00%)
- Formats de dates multiples : 4 formats differents detectes

---
*Rapport genere automatiquement - Audit de qualite des donnees de consommation énergetique des batiments*  
*Données brutes : consommations_raw.csv*  
*Script de traitement : 01_exploration_spark.ipynb*  
*Temps de traitement : 21.28 secondes*  
*Ressources machine approximatives allouées à ce traitement : (RAM : 11.09 GB, CPU : 84.30% - Approx logical cores free(13.49) - Approx physical cores free : 6.74)*  
*Date : 20/02/2026 20:29:56 (FR)* 



---

## Fermer la session Spark

In [17]:
spark.stop()

---

## (Optionnel) enregistrement dans un fichier temporaire du temps d'execution + ressources pour utilisation ultérieure (dans le script run_pipeline_hybride.py ou autres)

In [18]:
temps_resources = f"""
    Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)

    temps_exec_sec={temps_execution_01:.2f}
    ram_gb={ram_available_01:.2f}
    cpu_pct={cpu_available_pct_01:.2f}
    logi_cores={available_logical_01:.1f}
    physi_cores={available_physical_01:.1f}
"""

# Ecrire des données du temps d'execution + ressources dans le fichier TMP_FILE_TXT
TMP_FILE_TXT.write_text(temps_resources, encoding="utf-8")

138